# GoLLIE for Medical Entity Extraction

### Import requeriments
See the requeriments.txt file in the main directory to install the required dependencies


In [1]:
import sys
sys.path.append("../") # Add the GoLLIE base directory to sys path

In [2]:
import rich 
import logging
from src.model.load_model import load_model
import black
import inspect
from jinja2 import Template as jinja2Template
import tempfile
from src.tasks.utils_typing import AnnotationList
logging.basicConfig(level=logging.INFO)
from typing import Dict, List, Type

## Load GoLLIE

Load GOLLIE-7B from the huggingface-hub.
Use the AutoModelForCausalLM.from_pretrained function if you prefer it. However, creators provide a handy load_model function with many functionalities already implemented that will assist you in reproducing our results.

Please note that setting use_flash_attention=True is mandatory. Our flash attention implementation has small numerical differences compared to the attention implementation in Huggingface. Using use_flash_attention=False will result in the model producing inferior results. Flash attention requires an available CUDA GPU. Running GOLLIE pre-trained models on a CPU is not supported.

- Set force_auto_device_map=True to automatically load the model on available GPUs.
- Set quantization=4 if the model doesn't fit in your GPU memory.

In [3]:
#Use the custom load_model for loading GoLLIE
model, tokenizer = load_model(
    inference=True,
    model_weights_name_or_path="HiTZ/GoLLIE-7B",
    quantization=None,
    use_lora=False,
    force_auto_device_map=True,
    use_flash_attention=True,
    torch_dtype="bfloat16"
)

INFO:root:Loading model model from HiTZ/GoLLIE-7B
INFO:root:We will load the model using the following device map: auto and max_memory: None
INFO:root:Loading model with dtype: torch.bfloat16


>>>> Flash Attention installed
>>>> Flash RoPE installed


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

INFO:root:Model dtype: torch.bfloat16
INFO:root:Total model memory footprint: 13477.101762 MB


## Define the guidelines

First, we will define the labels and guidelines for the task. We will represent them as Python classes.

The following guidelines have been defined for this example. They were not part of the pre-training dataset. Therefore, we will run GOLLIE in zero-shot settings using unseen labels.

We will use the `Generic` class, which is a versatile class that allows for the implementation of any task you want. However, since the model has never seen the Generic label during training, we will rename it to Template, which is recognized by the model (as it was used in the Tacred dataset).

We will define several classes: `Illness`, `Medication`, `PatientData`, `HospitalizationData`. Each class will have a definition and a set of slots that the model needs to fill. Each slot also requires a type definition and a short description, which can include examples. For instance, for the `Illness` class, we define three slots:

- The `mention`, which will be the name of the Ilness of the patient and should be a string.
- The `treatment` which will be a list of treatments or interventions used to manage the illness. 
- The `symptoms`, which is defined as a list of symptoms. Therefore, GoLLIE will fill this slot with a list of strings.


In [18]:
from typing import List

from src.tasks.utils_typing import dataclass
from src.tasks.utils_typing import Generic as Template
from dataclasses import dataclass, field
"""
Entity definitions
"""


@dataclass
class Medication(Template):
    """Refers to a drug or substance used to diagnose, cure, treat, or prevent disease.
    Medications can be administered in various forms and dosages and are crucial 
    in managing patient health conditions. They can be classified based on their 
    therapeutic use, mechanism of action, or chemical characteristics."""
    
    mention: str
    """
    The name of the medication.
    Such as: "Aspirina", "Ibuprofeno", "Aspirina".
    """
    dosage: str # The amount and frequency at which the medication is prescribed. Such as: "100 mg al día", "200 mg dos veces al día"
    route: str # The method of administration for the medication. Such as: "oral", "intravenoso", "tópico"
    purpose: List[str]  # List of reasons or conditions for which the medication is prescribed. Such as: ["dolor", "control de azúcar en la sangre", "inflamación"]
    

@dataclass
class Ilness(Template):
    """Refers to a health condition or disease that affects the body's normal functioning.
    Illnesses can be caused by various factors, such as infections, genetic disorders,
    lifestyle choices, or environmental factors. They can affect different body systems
    and have varying degrees of severity."""
    
    mention: str
    """
    The name of the illness or health condition.
    Such as: "diabetes", "cáncer", "hipertensión".
    """
    symptoms: List[str] # List of signs or symptoms associated with the illness. Such as: ["dolor de cabeza", "fatiga", "fiebre"]
    treatment: List[str] # List of treatments or interventions used to manage the illness. Such as: ["medicamentos", "cirugía", "terapia física"]


@dataclass
class HospitalizationData:
    """Refers to information related to a patient's hospitalization, including the
    admission date, discharge date, and reason for hospitalization. Hospitalization
    data is essential for tracking patient health status, treatment progress, and
    healthcare resource utilization."""
    
    admission_date: str #The date on which the patient was admitted to the hospital.
    discharge_date: str #The date on which the patient was discharged from the hospital.
    reason: str #the reason or cause for the patient's hospitalization.
    
    
@dataclass
class PatientData:
    """Refers to information related to a patient's medical history, including
    name, age or urgency. Patient data is essential for healthcare providers 
    to provide appropriate care and make informed decisions about patient management."""
    
    name: str #The name of the patient.
    age: int #The age of the patient.
    urgency: str #The urgency level of the patient's condition.
    
    
    
ENTITY_DEFINITIONS: List[Template] = [
    Medication,
    Ilness,
    HospitalizationData,
]



if __name__ == "__main__":
    cell_txt = In[-1] #In needs to be imported from IPython


### Print the guidelines to guidelines.py

Due to IPython limitations, we must write the content of the previous cell to a file and then import the content from that file.

In [19]:
with open("guidelines.py","w",encoding="utf8") as python_guidelines:
    print(cell_txt,file=python_guidelines)

from guidelines import *    ### Print the guidelines to guidelines.py


We use inspect.getsource to get the guidelines as a string

In [20]:
guidelines = [inspect.getsource(definition) for definition in ENTITY_DEFINITIONS]
guidelines

['@dataclass\nclass Medication(Template):\n    """Refers to a drug or substance used to diagnose, cure, treat, or prevent disease.\n    Medications can be administered in various forms and dosages and are crucial \n    in managing patient health conditions. They can be classified based on their \n    therapeutic use, mechanism of action, or chemical characteristics."""\n    \n    mention: str\n    """\n    The name of the medication.\n    Such as: "Aspirina", "Ibuprofeno", "Aspirina".\n    """\n    dosage: str # The amount and frequency at which the medication is prescribed. Such as: "100 mg al día", "200 mg dos veces al día"\n    route: str # The method of administration for the medication. Such as: "oral", "intravenoso", "tópico"\n    purpose: List[str]  # List of reasons or conditions for which the medication is prescribed. Such as: ["dolor", "control de azúcar en la sangre", "inflamación"]\n',
 '@dataclass\nclass Ilness(Template):\n    """Refers to a health condition or disease tha

## Define input sentence

Here we define the input sentence and the gold labels.

You can define and empy list as gold labels if you don't have gold annotations.

In [21]:
import json
#import gt dataset
data = json.load(open('/hhome/nlp2_g09/Project/uab_summary_2024_with_gt.json'))

In [22]:
data[0]

{'Id': '1006949703',
 'Summary': '\n\n\n\nPaciente que ingresa por deterioro de estado general manifestado por disnea, astenia, edema y somnolencia con antecedentes de hipertensión arterial, EPOC, colitis isquémica, cirrosis hepática enólica, encefalopatía hepática y ERC. Durante el ingreso las pruebas exploratorias realizadas muestran cardiopatía hipertensiva, insuficiencia renal aguda de posible causa prerrenal e hipertensión portal sinuosidal. El diagnóstico se orienta a insuficiencia renal aguda en paciente con enfermedad renal cronica, sobrecarga de volúmen, insuficiencia respiratoria mixta y fiebre lo cual se decide tratar con diuréticos, antibióticos y oxigenoterapia con buena respuesta. Debido a la mejoría clínica del paciente  se procede al alta bajo seguimiento de especialistas y tratamiento farmacológico habitual. \n\n\n',
 'Text': " diagnostic alta codi icd-10 descripcio diagnostic k70.3 cirrosi hepatica alcoholica r18.8 altres tipus d'ascites dades informe motivo de ingres

In [23]:
text = data[0]['Text']
print(len(text))
gold = []

10287


## Filling a template

We need to define a template. For this task, we will include only the class definitions and the text to be annotated. However, we can design different templates to incorporate more information (for example, event triggers, as demonstrated in the Event Extraction notebook).

We will use Jinja templates, which are easy to implement and exceptionally fast. For more information, visit: https://jinja.palletsprojects.com/en/3.1.x/api/#high-level-api.



In [24]:
template_txt =(
"""# The following lines describe the task definition
{%- for definition in guidelines %}
{{ definition }}
{%- endfor %}

# This is the text to analyze
text = {{ text.__repr__() }}

# The annotation instances that take place in the text above are listed here
result = [
{%- for ann in annotations %}
    {{ ann }},
{%- endfor %}
]
""")


In [25]:

template = jinja2Template(template_txt)
# Fill the template
formated_text = template.render(guidelines=guidelines, text=text, annotations=gold, gold=gold)

### Black Code Formatter

We use the Black Code Formatter to automatically unify all the prompts to the same format. 

https://github.com/psf/black

In [26]:
black_mode = black.Mode()
formated_text = black.format_str(formated_text, mode=black_mode)

### Print the filled and formatted template

In [27]:
rich.print(formated_text)

# The following lines describe the task definition
@dataclass
class Medication(Template):
    """Refers to a drug or substance used to diagnose, cure, treat, or prevent disease.
    Medications can be administered in various forms and dosages and are crucial
    in managing patient health conditions. They can be classified based on their
    therapeutic use, mechanism of action, or chemical characteristics."""

    mention: str
    """
    The name of the medication.
    Such as: "Aspirina", "Ibuprofeno", "Aspirina".
    """
    dosage: str  # The amount and frequency at which the medication is prescribed. Such as: "100 mg al día", "200 
mg dos veces al día"
    route: str  # The method of administration for the medication. Such as: "oral", "intravenoso", "tópico"
    purpose: List[
        str
    ]  # List of reasons or conditions for which the medication is prescribed. Such as: ["dolor", "control de 
azúcar en la sangre", "inflamación"]


@dataclass
class Ilness(Template):
    """Refers to a health condition or disease that affects the body's normal functioning.
    Illnesses can be caused by various factors, such as infections, genetic disorders,
    lifestyle choices, or environmental factors. They can affect different body systems
    and have varying degrees of severity."""

    mention: str
    """
    The name of the illness or health condition.
    Such as: "diabetes", "cáncer", "hipertensión".
    """
    symptoms: List[
        str
    ]  # List of signs or symptoms associated with the illness. Such as: ["dolor de cabeza", "fatiga", "fiebre"]
    treatment: List[
        str
    ]  # List of treatments or interventions used to manage the illness. Such as: ["medicamentos", "cirugía", 
"terapia física"]


@dataclass
class HospitalizationData:
    """Refers to information related to a patient's hospitalization, including the
    admission date, discharge date, and reason for hospitalization. Hospitalization
    data is essential for tracking patient health status, treatment progress, and
    healthcare resource utilization."""

    admission_date: str  # The date on which the patient was admitted to the hospital.
    discharge_date: str  # The date on which the patient was discharged from the hospital.
    reason: str  # the reason or cause for the patient's hospitalization.


# This is the text to analyze
text = " diagnostic alta codi icd-10 descripcio diagnostic k70.3 cirrosi hepatica alcoholica r18.8 altres tipus 
d'ascites dades informe motivo de ingreso: mal estado general antecedentes personales: no ram. hta. disnea a 
pequeños esfuerzos y epoc gold ii en seguimiento en neumologia en tratamiento broncodilatador y oxigenoterapia en 
concentrador portatil a 3lx' para la deambulacion desde enero 2023. elevacion de hemidiafragma izquierdo y 
adenopatia hiliar derecha con captacion en pet-tc estudiado con ebus + fbs sin lesiones endobronquiales y con ap 
negativa para malignidad del bas y adenopatias puncionadas. hemorragia digestiva baja secundaria a colitis 
isquemica en setiembre 2022 (cordoba). colelitiasis. - cirrosis hepatica enolica child c en seguimiento en consulta
ch intensivo (dra. martin) **descompensacion edemo-ascitica en junio 2022 con empeoramiento tras ingreso 
hospitalario por colitis isquemica (setiembre 2022), en tratamiento diuretico con espironolactona 200mg/dia + 
furosemida 80mg/dia desde el 18.05.23 **encefalopatia hepatica en agosto 2022. en tratamiento con lactulosa + 
rifaximina desde noviembre 2022 **varices esofagicas pequeñas con signos de riesgo (fgs en setiembre - cordoba) en 
tratamiento con carvedilol 6.25mg/12horas **ultima ecografia en marzo 2023: no loes hepaticas, porta 
permeable,discreta cantidad de liquido libre perihepatico y en pelvis ex enolismo cronico, abstinente desde 
setiembre 2022. fumador 5-6 cig/dia tratamiento habitual: acfol 5mg/dia, carvedilol 6.25mg/12horas, espironolactona
200mg/dia, furosemida 80mg/dia, lactulosa, spiolto 2ing/12horas, iabvd. vive con su madr

## Prepare model inputs

We remove everything after `result =` to run inference with the model.

In [28]:
prompt, _ = formated_text.split("result =")
prompt = prompt + "result ="

Tokenize the input sentence

In [29]:
model_input = tokenizer(prompt, add_special_tokens=True, return_tensors="pt")

Remove the `eos` token from the input

In [30]:
model_input["input_ids"] = model_input["input_ids"][:, :-1]
model_input["attention_mask"] = model_input["attention_mask"][:, :-1]

## Run GoLLIE

We generate the predictions using GoLLIE.

We use `num_beams=1` and `do_sample=False` in our exmperiments. But feel free to experiment with differen decoding strategies 😊

In [33]:
import torch
print("CUDA Available: ", torch.cuda.is_available())
print("Number of GPUs: ", torch.cuda.device_count())
if torch.cuda.is_available():
    print("GPU Name:", torch.cuda.get_device_name(0))


CUDA Available:  False
Number of GPUs:  0


In [34]:
%%time


import torch
print(torch.cuda.device_count())
print(torch.cuda.get_device_name(3))  # Check if device 3 is available
device = "cuda:3" if torch.cuda.is_available() else "cpu"

model_ouput = model.generate(
    **model_input.to(model.device),
    max_new_tokens=128,
    do_sample=False,
    min_new_tokens=0,
    num_beams=1,
    num_return_sequences=1,
)


0


RuntimeError: No CUDA GPUs are available

### Print the results

In [35]:
for y, x in enumerate(model_ouput):
    print(f"Answer {y}")
    rich.print(tokenizer.decode(x,skip_special_tokens=True).split("result = ")[-1])

NameError: name 'model_ouput' is not defined

## Parse the output

The output is a Python list of instances, we can execute it  🤯

We define the AnnotationList class to parse the output with a single line of code. The `AnnotationList.from_output` function filters any label that we did not define (hallucinations) to prevent getting an `undefined class` error. 

result = AnnotationList.from_output(
    tokenizer.decode(model_ouput[0],skip_special_tokens=True).split("result = ")[-1],
    task_module="guidelines"
    )
rich.print(result)

In [36]:
type(result[0])

NameError: name 'result' is not defined

In [37]:
result[0].mention

NameError: name 'result' is not defined